In [5]:
# Try installing a known stable version (e.g., 1.3.0)
# If this fails, comment it out and try the next step.
!pip install pyBKT==1.3.0

# If the stable version didn't work, try installing the latest development version
# Uncomment the line below if the stable version install fails
# !pip install git+https://github.com/CAHLR/pyBKT.git

# After installation, you can add code to verify the installed version
!pip show pyBKT


Found existing installation: pyBKT 1.4.1
Uninstalling pyBKT-1.4.1:
  Successfully uninstalled pyBKT-1.4.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.4/32.4 MB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
학습 데이터:
    user_id      skill_name  correct
0  student1  Photosynthesis        0
1  student1  Photosynthesis        1
2  student1  Photosynthesis        1
3  student1         Gravity        0
4  student1         Gravity        1
5  student2  Photosynthesis        1
6  student2  Photosyn


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

AttributeError: _ARRAY_API not found

✗ 방법 1 실패: numpy.core.multiarray failed to import

방법 2: 수동 BKT 구현...

[Photosynthesis] 분석:
  student1: [0 1 1] → 최종 습득 확률 0.520
  student2: [1 1 0] → 최종 습득 확률 0.520

[Gravity] 분석:
  student1: [0 1] → 최종 습득 확률 0.194
  student2: [0 1] → 최종 습득 확률 0.194

간단한 BKT 분석 결과:
    user_id      skill_name  attempt  correct  estimated_mastery  \
0  student1  Photosynthesis        1        0            0.05085   
1  student1  Photosynthesis        2        1            0.19424   
2  student1  Photosynthesis        3        1            0.52034   
3  student2  Photosynthesis        1        1            0.65854   
4  student2  Photosynthesis        2        1            0.89668   
5  student2  Photosynthesis        3        0            0.52034   
6  student1         Gravity        1        0            0.05085   
7  student1         Gravity        2        1            0.19424   
8  student2         Gravity        1        0            0.05085   
9  student2         Gravity        2        1        

In [6]:
# Now, retry the original code block.
# It is recommended to run the entire cell again after the installation.
from pyBKT.models import Model
import pandas as pd
import numpy as np
import tempfile
import os

# 1. 학습 데이터 준비
data = pd.DataFrame([
    {'user_id': 'student1', 'skill_name': 'Photosynthesis', 'correct': 0},
    {'user_id': 'student1', 'skill_name': 'Photosynthesis', 'correct': 1},
    {'user_id': 'student1', 'skill_name': 'Photosynthesis', 'correct': 1},
    {'user_id': 'student1', 'skill_name': 'Gravity', 'correct': 0},
    {'user_id': 'student1', 'skill_name': 'Gravity', 'correct': 1},
    # 더 많은 데이터 추가 (BKT는 충분한 데이터가 필요)
    {'user_id': 'student2', 'skill_name': 'Photosynthesis', 'correct': 1},
    {'user_id': 'student2', 'skill_name': 'Photosynthesis', 'correct': 1},
    {'user_id': 'student2', 'skill_name': 'Photosynthesis', 'correct': 0},
    {'user_id': 'student2', 'skill_name': 'Gravity', 'correct': 0},
    {'user_id': 'student2', 'skill_name': 'Gravity', 'correct': 1},
])

print("학습 데이터:")
print(data)
print(f"데이터 형태: {data.shape}")
print("\n" + "="*60 + "\n")

try:
    # 방법 1: CSV 파일로 저장 후 파일 경로 전달 (pyBKT가 선호하는 방식)
    print("방법 1: CSV 파일을 통한 학습...")

    # 임시 CSV 파일 생성
    with tempfile.NamedTemporaryFile(mode='w', suffix='.csv', delete=False) as f:
        data.to_csv(f.name, index=False)
        temp_file = f.name

    try:
        model = Model(seed=42, num_fits=3)
        model.fit(data_path=temp_file) # Use data_path with the CSV file
        print("✓ 방법 1 성공: CSV 파일로 모델 학습 완료!")

        # 예측 수행
        # Pass the original DataFrame for prediction
        predictions = model.predict(data=data)
        print("\n예측 결과:")
        result_cols = ['user_id', 'skill_name', 'correct', 'correct_predictions', 'state_predictions']
        print(predictions[result_cols])

        # 스킬별 분석
        print("\n" + "="*50)
        print("스킬별 학습 진행도:")
        for skill in data['skill_name'].unique():
            skill_data = predictions[predictions['skill_name'] == skill]
            # Check if skill_data is not empty before accessing .iloc[0] and .iloc[-1]
            if not skill_data.empty:
                print(f"\n[{skill}]")
                print(f"- 초기 습득 확률: {skill_data['state_predictions'].iloc[0]:.3f}")
                print(f"- 최종 습득 확률: {skill_data['state_predictions'].iloc[-1]:.3f}")
                print(f"- 평균 정답률: {skill_data['correct'].mean():.3f}")
            else:
                print(f"\n[{skill}]: No data found for this skill.")


        # 모델 파라미터 출력
        print(f"\n학습된 BKT 파라미터:")
        if hasattr(model, 'params'):
            for skill in model.skills:
                if skill in model.params:
                    params = model.params[skill]
                    print(f"\n[{skill}]:")
                    for key, value in params.items():
                        if isinstance(value, (int, float)):
                            print(f"  {key}: {value:.4f}")

    finally:
        # 임시 파일 삭제
        if os.path.exists(temp_file):
            os.unlink(temp_file)

except Exception as e1:
    print(f"✗ 방법 1 실패: {str(e1)}")

    # The rest of the traceback handling and alternative methods
    # are kept as they were in the original code, but the primary fix
    # targets the pyBKT installation issue.
    try:
        # 방법 2: 다른 라이브러리 사용 (간단한 BKT 구현)
        print("\n방법 2: 수동 BKT 구현...")

        def simple_bkt_analysis(data):
            """간단한 BKT 스타일 분석"""
            results = []

            for skill in data['skill_name'].unique():
                skill_data = data[data['skill_name'] == skill].copy()
                skill_data = skill_data.sort_values(['user_id'])

                print(f"\n[{skill}] 분석:")

                for user in skill_data['user_id'].unique():
                    user_data = skill_data[skill_data['user_id'] == user]
                    responses = user_data['correct'].values

                    # 간단한 학습 진행도 계산
                    cumulative_correct = np.cumsum(responses)
                    total_attempts = np.arange(1, len(responses) + 1)
                    running_accuracy = cumulative_correct / total_attempts

                    # 베이지안 업데이트 시뮬레이션 (간단화)
                    prior = 0.3  # 초기 습득 확률
                    for i, correct in enumerate(responses):
                        # Ensure prior is within (0, 1) before updating
                        prior = max(1e-9, min(1 - 1e-9, prior))

                        if correct:
                            # Avoid division by zero or extreme values
                            denominator = prior * 0.9 + (1 - prior) * 0.2
                            if denominator > 1e-9:
                                prior = prior * 0.9 / denominator
                            else:
                                prior = 1.0 # Assume mastery if denominator is near zero and correct is 1
                        else:
                            denominator = prior * 0.1 + (1 - prior) * 0.8
                            if denominator > 1e-9:
                                prior = prior * 0.1 / denominator
                            else:
                                prior = 0.0 # Assume no mastery if denominator is near zero and correct is 0


                        results.append({
                            'user_id': user,
                            'skill_name': skill,
                            'attempt': i + 1,
                            'correct': correct,
                            'estimated_mastery': prior,
                            'running_accuracy': running_accuracy[i]
                        })

                    print(f"  {user}: {responses} → 최종 습득 확률 {prior:.3f}")

            return pd.DataFrame(results)

        # 간단한 BKT 분석 실행
        bkt_results = simple_bkt_analysis(data)

        print("\n" + "="*50)
        print("간단한 BKT 분석 결과:")
        print(bkt_results)

    except Exception as e2:
        print(f"✗ 방법 2도 실패: {str(e2)}")

        # 방법 3: 대안 라이브러리 제안
        print("\n" + "="*50)
        print("대안 해결책:")
        print("1. pyBKT 버전 문제일 가능성이 높습니다.")
        print("2. 다음 명령어들을 순서대로 시도해보세요:")
        print("   pip uninstall pyBKT")
        print("   pip install pyBKT==1.3.0")
        print("   또는")
        print("   pip install git+https://github.com/CAHLR/pyBKT.git")
        print("\n3. 다른 BKT 라이브러리 사용:")
        print("   - scikit-learn 기반 구현")
        print("   - TensorFlow/PyTorch 기반 구현")
        print("   - R의 BKT 패키지 사용")

        print("\n현재 데이터로 수동 계산된 학습 진행도:")
        for skill in data['skill_name'].unique():
            skill_data = data[data['skill_name'] == skill]
            accuracy = skill_data['correct'].mean()
            print(f"- {skill}: 정답률 {accuracy:.1%}")

except ImportError:
    print("ERROR: pyBKT 라이브러리가 설치되지 않았습니다.")
    print("설치 명령: pip install pyBKT")

학습 데이터:
    user_id      skill_name  correct
0  student1  Photosynthesis        0
1  student1  Photosynthesis        1
2  student1  Photosynthesis        1
3  student1         Gravity        0
4  student1         Gravity        1
5  student2  Photosynthesis        1
6  student2  Photosynthesis        1
7  student2  Photosynthesis        0
8  student2         Gravity        0
9  student2         Gravity        1
데이터 형태: (10, 3)


방법 1: CSV 파일을 통한 학습...



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

AttributeError: _ARRAY_API not found

✗ 방법 1 실패: numpy.core.multiarray failed to import

방법 2: 수동 BKT 구현...

[Photosynthesis] 분석:
  student1: [0 1 1] → 최종 습득 확률 0.520
  student2: [1 1 0] → 최종 습득 확률 0.520

[Gravity] 분석:
  student1: [0 1] → 최종 습득 확률 0.194
  student2: [0 1] → 최종 습득 확률 0.194

간단한 BKT 분석 결과:
    user_id      skill_name  attempt  correct  estimated_mastery  \
0  student1  Photosynthesis        1        0            0.05085   
1  student1  Photosynthesis        2        1            0.19424   
2  student1  Photosynthesis        3        1            0.52034   
3  student2  Photosynthesis        1        1            0.65854   
4  student2  Photosynthesis        2        1            0.89668   
5  student2  Photosynthesis        3        0            0.52034   
6  student1         Gravity        1        0            0.05085   
7  student1         Gravity        2        1            0.19424   
8  student2         Gravity        1        0            0.05085   
9  student2         Gravity        2        1        